## Introduction

In `Provision Package Preparation` step of the README, we created `audit.pkl` and `zip` files for all the provisioned parties (server, clients, and admins) in `expr_files/`. The zip files are encrypted and the passwords are saved in `audit.pkl`.

In an experiment, you need to send decrypted folders to each site so they could run it on their system. Therefore, in this notebook, we would decrypt and send folders to all the provisioned parties. After running this notebook.

In [1]:
import shutil
from zipfile import ZipFile
import pickle
import os

In [2]:
os.listdir("expr_files/")

['project.yml',
 'prerpare_expr_files.sh',
 'researcher@nvidia.com.zip',
 'download_dataset.py',
 'authz_config.json',
 'org1-b.zip',
 'researcher@org2.com.zip',
 'admin@nvidia.com.zip',
 'org1-a.zip',
 'audit.pkl',
 'server.zip',
 'researcher@org1.com.zip',
 'org2.zip',
 'it@org2.com.zip']

In this example, `server.zip` will be used to create the server, `org1-a.zip` and `org1-b.zip` will be used to create two clients, and `admin@nvidia.com.zip` will be used to create an admin client to operate the FL experiment.

First, unzip all the packages with the following code:

In [3]:
startup_path = "expr_files"  # this is the path that contains `audit.pkl` and zip files
workspace = "demo_workspace"  # this is the folder that will be created to contain all experiment related files

if not os.path.exists(workspace):
    os.makedirs(workspace)
    print(workspace, " created!")

admin_name = "admin@nvidia.com"
client_name_1 = "org1-a"
client_name_2 = "org1-b"
server_name = "server"

# access audit file get passwords for unzipping packages
with open(os.path.join(startup_path, "audit.pkl"), 'rb') as handle:
    audit_file = pickle.load(handle)

proj_name = list(audit_file.keys())[0]
pw_key = "zip_pw"
server_folder_list = ["server"]
client_folder_list = ["admin_clients", "fl_clients"]

folder_pwd_dict = {}
for obj in server_folder_list:
    unzip_pw = audit_file[proj_name][obj][pw_key]
    folder_pwd_dict[obj] = unzip_pw

for obj in client_folder_list:
    obj_sub_dict = audit_file[proj_name][obj]
    for client in obj_sub_dict.keys():
        unzip_pw = obj_sub_dict[client][pw_key]
        folder_pwd_dict[client] = unzip_pw

# unzip all folders into workspace
for name, pwd in folder_pwd_dict.items():
    zip_file_path = os.path.join(startup_path, name + ".zip")
    dst_file_path = os.path.join(workspace, name)
    if not os.path.exists(dst_file_path):
        os.makedirs(dst_file_path)
    with ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(path=dst_file_path, pwd=bytes(pwd, 'utf-8'))
    # change permissions
    if ".com" in name:
        sub_file_list = ["docker.sh", "fl_admin.sh"]
    else:
        sub_file_list = ["start.sh", "sub_start.sh", "docker.sh"]
    for file in sub_file_list:
        os.chmod(os.path.join(dst_file_path, "startup", file), 0o755)
    print("unzip: {} finished.".format(name))

demo_workspace  created!
unzip: server finished.
unzip: admin@nvidia.com finished.
unzip: researcher@nvidia.com finished.
unzip: researcher@org1.com finished.
unzip: researcher@org2.com finished.
unzip: it@org2.com finished.
unzip: org1-a finished.
unzip: org1-b finished.
unzip: org2 finished.


In [4]:
# check the created workspace
os.listdir(workspace)

['org1-b',
 'researcher@nvidia.com',
 'server',
 'admin@nvidia.com',
 'researcher@org2.com',
 'org1-a',
 'org2',
 'researcher@org1.com',
 'it@org2.com']

With default settings, the experiment related config folder `spleen_example` should be copied into the `transfer` folder within the admin package:

In [5]:
config_folder = "spleen_example"

transfer_path = os.path.join(workspace, admin_name, "transfer/")
if not os.path.exists(transfer_path):
    os.makedirs(transfer_path)
shutil.copytree(config_folder, os.path.join(transfer_path, config_folder))
print("copied {} into {}.".format(config_folder, transfer_path))

copied spleen_example into demo_workspace/admin@nvidia.com/transfer/.


So far, all required files are created in the workspace. Before starting the docker images, we can update the permissions for these files:

In [6]:
!chown -R 1000:1000 demo_workspace/*

### Next Steps

You have now finished unzipping the provisioning files and copying the experiment folder to the admin's transfer folder.
In the next notebook, [Server Startup Notebook](2-Server.ipynb), you will start the server container.